In [2]:
import pandas as pd 

In [3]:
from aind_data_access_api.document_db import MetadataDbClient

API_GATEWAY_HOST = "api.allenneuraldynamics.org"
DATABASE = 'metadata_index'
COLLECTION = 'data_assets'

docdb_api_client = MetadataDbClient(
   host=API_GATEWAY_HOST,
   database=DATABASE,
   collection=COLLECTION,
)
print(docdb_api_client._base_url)

https://api.allenneuraldynamics.org/v1/metadata_index/data_assets


In [4]:
aggregate = [
    {
        "$match": {
            "session.session_start_time": {"$regex": "^2025"},
            "data_description.modality.abbreviation": {"$nin": ["ecephys", "fib"]},
            "data_description.data_level": "derived",
            "data_description.project_name": "Behavior Platform",
            "procedures": {"$ne": None},
            "$and": [
                {"quality_control.evaluations": {"$exists": True, "$ne": []}},
                {"quality_control.evaluations": {
                    "$not": {
                        "$elemMatch": { "latest_status": { "$ne": "Pass" } }
                    }
                }}
            ]
        }
    },
    {
        "$project": {
            "name": 1,
            "subject_id": "$data_description.subject_id",
            "genotype": "$subject.genotype",
            "date_of_birth": "$subject.date_of_birth",
            "sex": "$subject.sex",
            "session_type": "$session.session_type",
            "session_time": "$session.session_start_time",
            "stimulus_epochs": "$session.stimulus_epochs.stimulus_name",
            "project_name": "$data_description.project_name", 
            "modality": "$data_description.modality.name",
            "trials_total": "$session.stimulus_epochs.trials_total",
            "trials_rewarded": "$session.stimulus_epochs.trials_rewarded",
            "quality_control": {
                "$map": {
                    "input": "$quality_control.evaluations",
                    "as": "eval",
                    "in": "$$eval.latest_status"
                }
            }
        }
    }
]

records = docdb_api_client.aggregate_docdb_records(
    pipeline=aggregate
)


In [5]:
df = pd.DataFrame(records)
df = df.drop_duplicates(subset='name')
df

,_id,name,subject_id,genotype,date_of_birth,sex,session_type,session_time,stimulus_epochs,project_name,modality,trials_total,trials_rewarded,quality_control
0,8bcbd349-2ac7-467a-8ccd-f05b2319f06d,behavior_761432_2025-03-03_11-51-23_processed_...,761432,Dbh-Cre-KI/wt,2024-08-21,Male,Coupled Baiting,2025-03-03T11:51:23.846831-08:00,[The behavior auditory go cue],Behavior Platform,[Behavior],[613],[246],"[Pass, Pass, Pass, Pass, Pass, Pass]"
1,d9dce157-83e7-4d79-8c42-c48a73d7849b,behavior_761433_2025-03-03_11-47-13_processed_...,761433,Dbh-Cre-KI/wt,2024-08-21,Male,Coupled Without Baiting,2025-03-03T11:47:13.392064-08:00,[The behavior auditory go cue],Behavior Platform,[Behavior],[509],[209],"[Pass, Pass, Pass, Pass, Pass, Pass]"
2,ed9f660f-0990-427f-84cf-5d454331c348,behavior_770803_2025-02-28_13-11-33_processed_...,770803,Ai230(TIT2L-XCaMPG-WPRE-ICL-ChRmine-oScarlet-I...,2024-10-07,Male,Uncoupled Baiting,2025-02-28T13:11:33.083835-08:00,[The behavior auditory go cue],Behavior Platform,[Behavior],[539],[261],"[Pass, Pass, Pass, Pass, Pass, Pass]"
3,c1d96ba0-2393-45b4-a835-f06d8846d8ed,behavior_781469_2025-03-03_13-26-23_processed_...,781469,wt/wt,2024-10-08,Male,Uncoupled Baiting,2025-03-03T13:26:23.690824-08:00,[The behavior auditory go cue],Behavior Platform,[Behavior],[403],[180],"[Pass, Pass, Pass, Pass, Pass, Pass]"
4,7de28343-b4f0-466f-912c-a5792de1e28a,behavior_752014_2025-03-03_09-34-48_processed_...,752014,Dbh-Cre-KI/wt,2024-07-05,Male,Uncoupled Without Baiting,2025-03-03T09:34:48.766920-08:00,[The behavior auditory go cue],Behavior Platform,[Behavior],[651],[333],"[Pass, Pass, Pass, Pass, Pass, Pass]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,b7ebd2ba-c5fe-4af8-bba0-8220db7290df,behavior_758018_2025-02-18_09-19-22_processed_...,758018,Dbh-Cre-KI/wt,2024-08-04,Female,Uncoupled Without Baiting,2025-02-18T09:19:22.803530-08:00,[The behavior auditory go cue],Behavior Platform,[Behavior],[559],[297],"[Pass, Pass, Pass, Pass, Pass, Pass]"
425,a2c65035-89b6-4b7f-a857-fd9e1f45fcaa,behavior_777098_2025-04-22_18-17-36_processed_...,777098,Drd1a-Cre/wt;Adora2a-T2A-FlpO/wt;Ai233(TICL-Ch...,2024-11-11,Female,Uncoupled Baiting,2025-04-22T18:17:36.949337-07:00,[The behavior auditory go cue],Behavior Platform,[Behavior],[466],[161],"[Pass, Pass, Pass, Pass, Pass]"
426,4cf4a354-1128-4965-bab1-a7d5963076a2,behavior_777098_2025-04-21_16-01-17_processed_...,777098,Drd1a-Cre/wt;Adora2a-T2A-FlpO/wt;Ai233(TICL-Ch...,2024-11-11,Female,Uncoupled Baiting,2025-04-21T16:01:17.308796-07:00,[The behavior auditory go cue],Behavior Platform,[Behavior],[460],[210],"[Pass, Pass, Pass, Pass, Pass]"
427,651628ee-e241-4427-8b7d-962bba608e01,behavior_774390_2025-06-25_18-42-52_processed_...,774390,Drd3-Cre_KI196/wt,2024-10-26,Male,Uncoupled Baiting,2025-06-25T18:42:52.752822-07:00,[The behavior auditory go cue],Behavior Platform,[Behavior],[496],[231],"[Pass, Pass, Pass, Pass, Pass]"


In [7]:
df.to_csv('/data/metadata/dynamic_foraging_metadata.csv', index= False)